In [1]:
#Import drive
from google.colab import drive
#Mount Google Drive
drive.mount("/content/drive")

Mounted at /content/drive


In [1]:
%cd drive/MyDrive/Colab Notebooks/Sebastian

/content/drive/MyDrive/Colab Notebooks/Sebastian


Stuff for commiting to Github below

In [4]:
ROOT = '/content/drive"'

In [44]:
from os.path import join  

# path to your project on Google Drive
MY_GOOGLE_DRIVE_PATH = 'drive/MyDrive/Colab Notebooks/Sebastian' 
# replace with your Github username 
GIT_USERNAME = "ahhuang007" 
# definitely replace with your
GIT_TOKEN = "github token here"  
# Replace with your github repository in this case we want 
# to clone deep-learning-v2-pytorch repository
GIT_REPOSITORY = "Sebastian" 

PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH)

# It's good to print out the value if you are not sure 
print("PROJECT_PATH: ", PROJECT_PATH)   

# In case we haven't created the folder already; we will create a folder in the project path     

#GIT_PATH = "https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git" this return 400 Bad Request for me
#GIT_PATH = "https://" + GIT_TOKEN + "@github.com/" + GIT_USERNAME + "/" + GIT_REPOSITORY + ".git"
#print("GIT_PATH: ", GIT_PATH)

PROJECT_PATH:  /content/drive"/drive/MyDrive/Colab Notebooks/Sebastian


In [ ]:
!git add .

In [56]:
!git commit -m "got my gym environment to register, still need to fix the timestep error"

[main 420a3af] test github commit
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite colab.ipynb (94%)


In [20]:
!git config --global user.email "ahhuang007@gmail.com"
!git config --global user.name "ahhuang007"

fatal: remote origin already exists.


Had to reset the remote origin URL here, removed the code since it contained the personal authorization token.

In [57]:
!git push origin

Counting objects: 3, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 1.23 KiB | 251.00 KiB/s, done.
Total 3 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/ahhuang007/Sebastian
   52d37e8..420a3af  main -> main


In [48]:
print("testing github stuff")

testing github stuff


Getting OpenAI Baselines

In [ ]:
!git clone https://github.com/openai/baselines.git

Cloning into 'baselines'...
remote: Enumerating objects: 3627, done.
remote: Total 3627 (delta 0), reused 0 (delta 0), pack-reused 3627
Receiving objects: 100% (3627/3627), 6.46 MiB | 8.34 MiB/s, done.
Resolving deltas: 100% (2429/2429), done.
Checking out files: 100% (199/199), done.


Baselines doesn't come installed, tragic

In [5]:
cd baselines

/content/drive/MyDrive/Colab Notebooks/Sebastian/baselines


In [ ]:
!pip install -e .

Installing stable_baselines since it might be better

In [53]:
%cd ..

/content/drive/My Drive/Colab Notebooks


In [3]:
!pip install stable-baselines3[extra]

     |████████████████████████████████| 161 kB 5.4 MB/s 


Installing my custom environment

In [54]:
cd Sebastian

/content/drive/My Drive/Colab Notebooks/Sebastian


In [6]:
%cd gym-seb

/content/drive/My Drive/Colab Notebooks/Sebastian/gym-seb


In [7]:
!pip install -e .

Obtaining file:///content/drive/My%20Drive/Colab%20Notebooks/Sebastian/gym-seb
  Attempting uninstall: gym-seb
    Found existing installation: gym-seb 0.0.2
    Can't uninstall 'gym-seb'. No files were found to uninstall.
  Running setup.py develop for gym-seb


Restart runtime here

In [1]:
%cd drive/My Drive/Colab Notebooks/Sebastian

/content/drive/My Drive/Colab Notebooks/Sebastian


In [2]:
from stable_baselines3 import DDPG
import gym

Copying/pasting the environment here, because it's what they do in documentation and I'm stuck

In [7]:
from gym import error, spaces, utils
from gym.utils import seeding
import pybullet as p
import time
import pybullet_data
import numpy as np
import pandas as pd

class SebEnv(gym.Env):
  metadata = {'render.modes': ['human']}

  def __init__(self,
               urdf_root=pybullet_data.getDataPath(),
               distance_limit=float("inf"),
               self_collision_enabled=True,
               motor_velocity_limit=np.inf,
               pd_control_enabled=False,
               leg_model_enabled=True,
               accurate_motor_model_enabled=False,
               remove_default_joint_damping=False,
               motor_kp=1.0,
               motor_kd=0.02,
               control_latency=0.0,
               pd_latency=0.0,
               torque_control_enabled=False,
               motor_overheat_protection=False,
               hard_reset=True,
               on_rack=False,
               render=False,
               num_steps_to_log=1000,
               action_repeat=1,
               control_time_step=None,
               env_randomizer=None,
               forward_reward_cap=float("inf"),
               reflection=True,
               log_path=None):
    physicsClient = p.connect(p.DIRECT)#or p.DIRECT for non-graphical version
    p.setAdditionalSearchPath(pybullet_data.getDataPath()) #optionally
    p.setGravity(0,0,-10)
    p.resetDebugVisualizerCamera(cameraDistance = 1.5, cameraYaw=0, cameraPitch=0, cameraTargetPosition=[0,0,0])
    planeId = p.loadURDF("plane.urdf")
    
    self.cubeStartPos = [0,0,0.1]
    
    self.cubeStartOrientation = p.getQuaternionFromEuler([0,0,0])
    
    self.boxId = p.loadURDF("sebastian_v2.urdf",self.cubeStartPos, self.cubeStartOrientation, 
                       flags = p.URDF_USE_SELF_COLLISION_EXCLUDE_PARENT)
    
    self.mode = p.POSITION_CONTROL
    self.joints = [1,2,3,5,6,7,9,10,11,13,14,15]

    cubePos, cubeOrn = p.getBasePositionAndOrientation(self.boxId)
    
    self.action_space = spaces.Box(np.array([-1.5708]*12), np.array([+1.5708]*12), dtype = np.float32)
    self.observation_space = spaces.Box(np.array([-100000]*3), np.array([+100000]*3))
    
  def step(self, action):
    p.stepSimulation()
    op, oo = p.getBasePositionAndOrientation(self.boxId)
    
    pos = action#.numpy()
    
    p.setJointMotorControlArray(self.boxId, self.joints, controlMode=self.mode, targetPositions=pos)
    #time.sleep(1./25.)
    nep, no = p.getBasePositionAndOrientation(self.boxId)
    observation = nep
    reward = (nep[0] - np.abs(op[0])) - np.abs(op[1])
    print(nep)
    info = {}
    done = False
    if nep[0] > 0:
      done = True
    return np.array(observation), reward, done, info

  def reset(self):
    p.resetBasePositionAndOrientation(self.boxId, self.cubeStartPos, self.cubeStartOrientation)
    p.resetBaseVelocity(self.boxId, [0, 0, 0], [0, 0, 0])
    position, ori = p.getBasePositionAndOrientation(self.boxId)
    return np.array(position)
    
  def render(self, mode='human'):
    ...
  def close(self):
    p.disconnect()

In [3]:
import gym_seb
env = gym.make('seb-v0')

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [4]:
from stable_baselines3.common.env_checker import check_env

check_env(env, warn=True)

(-9.135539262493242e-22, 1.5455676224887002e-20, 0.0998263888888889)
(-0.00029545139535407404, -7.149146799191966e-05, 0.09935338029742974)
(-0.0004736046119223287, -0.0001250302059447957, 0.09879949809695165)
(-0.0006373498854392942, -0.0002227233121884425, 0.09819576978197853)
(-0.0007737970422993372, -0.00027275655499671994, 0.09732979494833223)
(-0.0008695059656710828, -0.00020774131823518116, 0.09619908565635063)
(-0.000869728057248855, -0.00017287746386868505, 0.09511400666486133)
(-0.0009378923264779374, -0.00034464216433709204, 0.09383770456346552)
(-0.000836177673188474, -0.00022291539934467133, 0.09222652379197531)
(-0.0007580464250242059, -0.0001415050945045844, 0.09044855920854708)
(-0.0008728176941297606, -0.00025040670893109493, 0.08845268706451807)


/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/env_checker.py:273: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  "We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) "


In [5]:
from stable_baselines3.ddpg.policies import MlpPolicy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from stable_baselines3 import DDPG

model = DDPG('MlpPolicy', env, verbose = 1).learn(100)

Streaming output truncated to the last 5000 lines.
(-0.237905045004437, 0.02030769325135955, 0.04596849215511471)
(-0.23793931686356176, 0.02027145669057031, 0.04597110415161076)
(-0.23797416421550566, 0.02023348013356294, 0.0459737391224119)
(-0.23800950872770163, 0.02019384999871708, 0.04597638330963875)
(-0.23804527311015203, 0.020152664930020346, 0.04597902519837073)
(-0.23808137620613287, 0.020110031637985328, 0.04598165250466396)
(-0.23811773697915037, 0.02006606594295736, 0.0459842516856348)
(-0.23815427791082006, 0.020020889398836852, 0.04598681241205397)
(-0.23819091810070478, 0.019974627312127756, 0.04598932415214684)
(-0.23822757571973693, 0.01992741171074906, 0.04599177380782077)
(-0.23826417013789888, 0.01987937997799885, 0.04599414754719673)
(-0.2383006217234652, 0.019830673917581775, 0.04599643177551654)
(-0.23833695164975, 0.019781393954581023, 0.045998560951925144)
(-0.23837306184280796, 0.019731637274192908, 0.04600051231939634)
(-0.2384088588000042, 0.019681530995262

KeyboardInterrupt: ignored